# SEGMENTACIÓN DE IMÁGENES MÉDICAS: CANALES RADICULARES

## Autor: Urko Alli Barrena

In [ ]:
!nvidia-smi
!pip install ultralytics roboflow opencv-python pandas

Sat Mar 29 16:06:10 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   51C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
import os
import zipfile
import pandas as pd
import shutil
from google.colab import drive, files
from ultralytics import YOLO
import json

def montar_drive():
    """
    Objetivo:
        Permite cargar la cuenta de Google Drive en el entorno de ejecución de Google Colab.

    Parámetros:
        None: no devuelve nada.

    Returns:
        None: no devuelve nada.
    """
    drive.mount('/content/drive')

def extraer_datasets(ruta_archivo_zip, directorio_salida):
    """
    Objetivo:
        Permite extraer el contenido de un archivo ZIP que almacena los datasets en un directorio específico.

    Parámetros:
        ruta_zip (str): ruta del archivo ZIP que se quiere extraer.
        directorio_salida (str): directorio donde se almacenarán los datasets una vez descomprimida la carpeta que los almacena.

    Returns:
        None: no devuelve nada.
    """
    with zipfile.ZipFile(ruta_archivo_zip, 'r') as archivo_zip:
        archivo_zip.extractall(directorio_salida)

def entrenamiento_validacion(directorio_datasets, indice_dataset):
    """
    Objetivo:
        Permite realizar el entrenamiento y la validación del modelo YOLO11 para segmentar las imágenes almacenadas en los datasets extraídos previamente.

    Parámetros:
        directorio_datasets (str): ruta del directorio donde se encuentran los datasets almacenados.
        indice_dataset (int): índice del dataset que se va a utilizar para realizar el entrenamiento.

    Returns:
        None: no devuelve nada.
    """
    ruta_data_yaml = os.path.join(directorio_datasets, f'dataset{indice_dataset}.yaml')
    model = YOLO('yolo11n-seg.pt')
    model.train(task='segment', mode='train', data=ruta_data_yaml, epochs=50, val=True, patience=50, augment=True, imgsz=1024, save_json=True)

def obtener_datasets(directorio_datasets):
    """
    Objetivo:
        Permite obtener, y a la vez ordenar las rutas de los datasets en un directorio.

    Parámetros:
        directorio_datasets (str): ruta del directorio donde se almacenan los datasets.

    Returns:
        list: lista con las rutas de los datasets ordenados según su índice.
    """
    return sorted([os.path.join(directorio_datasets, i) for i in os.listdir(directorio_datasets) if os.path.isdir(os.path.join(directorio_datasets, i))],
        key=lambda x: int(x.split('Dataset')[-1]))

def ejecutar_entrenamiento(directorio_datasets):
    """
    Objetivo:
        Permite ejecutar el proceso completo de entrenamiento y validación para cada dataset.

    Parámetros:
        directorio_datasets (str): ruta del directorio donde se almacenan los datasets.

    Returns:
        None: no devuelve nada.
    """
    datasets = obtener_datasets(directorio_datasets)
    for indice_dataset, dataset in enumerate(datasets, start=1):
        entrenamiento_validacion(dataset, indice_dataset)

def encontrar_mejor_modelo(ruta_ejecucion="/content/runs/segment/"):
    """
    Objetivo:
        Permite encontrar el mejor modelo entrenado basándose en la métrica mAP50-95.

    Parámetros:
        runs_path (str): ruta donde se almacenan las carpetas de entrenamiento generadas por YOLO.

    Returns:
        str or None: ruta del mejor modelo encontrado o None si no se encuentra un modelo válido.
    """
    carpetas_entrenamiento = sorted(
        [archivo for archivo in os.listdir(ruta_ejecucion) if archivo.startswith("train") and os.path.isdir(os.path.join(ruta_ejecucion, archivo))],
        key=lambda x: int(x.replace("train", "")) if x.replace("train", "").isdigit() else 0
    )
    mejor_mAP = 0
    mejor_modelo = None
    for carpeta in carpetas_entrenamiento:
        carpeta_entrenamiento = os.path.join(ruta_ejecucion, carpeta)
        csv_resultados = os.path.join(carpeta_entrenamiento, "results.csv")
        ruta_mejor_modelo = os.path.join(carpeta_entrenamiento, "weights", "best.pt")
        if os.path.exists(csv_resultados) and os.path.exists(ruta_mejor_modelo):
            resultados = pd.read_csv(csv_resultados)
            if "metrics/mAP50-95(M)" in resultados.columns:
                max_mAP = resultados["metrics/mAP50-95(M)"].max()
                if max_mAP > mejor_mAP:
                    mejor_mAP = max_mAP
                    mejor_modelo = ruta_mejor_modelo
    return mejor_modelo

def predecir_con_mejor_modelo(ruta_mejor_modelo, directorio_datasets):
    """
    Objetivo:
        Permite utilizar el mejor modelo entrenado para realizar las predicciones. Además, permite obtener las
        imágenes con los contornos predichos, y las coordenadas de dichos contornos.

    Parámetros:
        ruta_mejor_modelo (str): ruta del mejor modelo entrenado.
        directorio_datasets (str): ruta de los datasets con las imagenes de test.

    Returns:
        None: no devuelve nada
    """
    model = YOLO(ruta_mejor_modelo)
    diccionario_predicciones = {}
    for numero_dataset in range(1, 6):
        ruta_imagenes_test = os.path.join(directorio_datasets, f'Dataset{numero_dataset}/test/images')
        resultados = model.predict(ruta_imagenes_test, save=True, save_txt=True)
        for resultado in resultados:
            nombre_archivo = os.path.basename(resultado.path)
            if hasattr(resultado, 'boxes') and resultado.boxes is not None:
                coordenadas = resultado.boxes.xyxy.tolist()
            elif hasattr(resultado, 'masks') and resultado.masks is not None:
                coordenadas = resultado.masks.xy.tolist()
            else:
                coordenadas = []
            diccionario_predicciones[nombre_archivo] = coordenadas


def comprimir_y_descargar_resultados(carpeta_salida):
    """
    Objetivo:
        Permite comprimir los resultados de las predicciones en un archivo ZIP y lo descarga automáticamente.

    Parámetros:
        carpeta_salida: nombre del directorio o carpeta en el que se almacenarán los resultados.

    Returns:
        None: no devuelve nada.
    """
    shutil.make_archive(f'/content/{carpeta_salida}', 'zip', '/content/runs/segment')
    files.download(f'/content/{carpeta_salida}.zip')

'''
# Vincular con Google Drive
montar_drive()

# Tras haber metido los Datasets (comprimidos) en la carpeta TFG de mi Drive.
ruta_zip = '/content/drive/MyDrive/TFG/Datasets_Comprimidos.zip'

# Ruta donde se almacenarán los datasets extraídos
directorio_datasets = '/content/Datasets'

# Extracción de datasets
extraer_datasets(ruta_zip, directorio_datasets)

# Ejecución del entrenamiendo-validación del modelo.
ejecutar_entrenamiento(directorio_datasets)

# Obtencion de la ruta del mejor modelo entrenado (por defecto la ruta donde se almacenan siempre)
ruta_mejor_modelo = encontrar_mejor_modelo()
print (ruta_mejor_modelo)

# Predicciones
predecir_con_mejor_modelo(ruta_mejor_modelo, directorio_datasets)

# Resultados finales
comprimir_y_descargar_resultados("50_iteraciones")
'''

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


'\n# Vincular con Google Drive\nmontar_drive()\n\n# Tras haber metido los Datasets (comprimidos) en la carpeta TFG de mi Drive.\nruta_zip = \'/content/drive/MyDrive/TFG/Datasets_Comprimidos.zip\'\n\n# Ruta donde se almacenarán los datasets extraídos\ndirectorio_datasets = \'/content/Datasets\'\n\n# Extracción de datasets\nextraer_datasets(ruta_zip, directorio_datasets)\n\n# Ejecución del entrenamiendo-validación del modelo.\nejecutar_entrenamiento(directorio_datasets)\n\n# Obtencion de la ruta del mejor modelo entrenado (por defecto la ruta donde se almacenan siempre)\nruta_mejor_modelo = encontrar_mejor_modelo()\nprint (ruta_mejor_modelo)\n\n# Predicciones\npredecir_con_mejor_modelo(ruta_mejor_modelo, directorio_datasets)\n\n# Resultados finales\ncomprimir_y_descargar_resultados("50_iteraciones")\n'

In [ ]:
# Vincular con Google Drive
montar_drive()

# Tras haber metido los Datasets (comprimidos) en la carpeta TFG de mi Drive.
dataset = '/content/drive/MyDrive/TFG/Datasets_Comprimidos.zip'
entrenamiento1 = '/content/drive/MyDrive/TFG/SEG_VC_5FOLDS_PRUEBA.zip'
entrenamiento2 = '/content/drive/MyDrive/TFG/SEG_VC_5FOLD_200ITER.zip'
entrenamiento3 = '/content/drive/MyDrive/TFG/SEG_VC_5FOLD_350ITER.zip'
entrenamiento4 = '/content/drive/MyDrive/TFG/SEG_VC_5FOLD_500ITER.zip'


# Ruta donde se almacenarán los datasets extraídos
directorio_datasets = '/content/Datasets'
directorio_entrenamientos = '/content/Entrenamientos'

# Extracción de datasets
extraer_datasets(dataset, directorio_datasets)
extraer_datasets(entrenamiento1, directorio_entrenamientos)
extraer_datasets(entrenamiento2, directorio_entrenamientos)
extraer_datasets(entrenamiento3, directorio_entrenamientos)
extraer_datasets(entrenamiento4, directorio_entrenamientos)

Mounted at /content/drive


In [ ]:
rutas_mejor_modelo = []
for carpetas in os.listdir(directorio_entrenamientos):
  for archivo in os.listdir(os.path.join(directorio_entrenamientos,carpetas)):
    mejor_modelo_actual = encontrar_mejor_modelo(os.path.join(directorio_entrenamientos,carpetas))
  rutas_mejor_modelo.append(mejor_modelo_actual)
print (rutas_mejor_modelo)

['/content/Entrenamientos/SEG_VC_5FOLD_200ITER/train4/weights/best.pt', '/content/Entrenamientos/SEG_VC_5FOLDS_PRUEBA/train4/weights/best.pt', '/content/Entrenamientos/SEG_VC_5FOLD_500ITER/train4/weights/best.pt', '/content/Entrenamientos/SEG_VC_5FOLD_350ITER/train4/weights/best.pt']


In [ ]:
resultados_posibles = [ruta.split('train4/')[0] + 'train4/results.csv' for ruta in rutas_mejor_modelo]

mejor_mAP = 0
mejor_mAP_ruta = ""
for ruta in resultados_posibles:
  resultados = pd.read_csv(ruta)
  if "metrics/mAP50-95(M)" in resultados.columns:
      max_mAP = resultados["metrics/mAP50-95(M)"].max()
      if max_mAP > mejor_mAP:
          mejor_mAP = max_mAP
          mejor_mAP_ruta = ruta

In [ ]:
predecir_con_mejor_modelo(ruta.split('train4/')[0] + 'train4/weights/best.pt', directorio_datasets)


image 1/9 /content/Datasets/Dataset1/test/images/CASO_63.png: 1024x800 1 ROOT, 1 TOOTH, 16.2ms
image 2/9 /content/Datasets/Dataset1/test/images/CASO_65.png: 1024x800 1 ROOT, 1 TOOTH, 15.7ms
image 3/9 /content/Datasets/Dataset1/test/images/CASO_66.png: 1024x800 1 ROOT, 1 TOOTH, 15.7ms
image 4/9 /content/Datasets/Dataset1/test/images/CASO_71.png: 1024x800 1 ROOT, 1 TOOTH, 15.7ms
image 5/9 /content/Datasets/Dataset1/test/images/CASO_72.png: 1024x768 1 ROOT, 1 TOOTH, 16.2ms
image 6/9 /content/Datasets/Dataset1/test/images/CASO_73.png: 1024x864 1 ROOT, 1 TOOTH, 18.4ms
image 7/9 /content/Datasets/Dataset1/test/images/CASO_78.png: 1024x800 1 ROOT, 1 TOOTH, 18.7ms
image 8/9 /content/Datasets/Dataset1/test/images/CASO_85.png: 1024x800 1 ROOT, 1 TOOTH, 15.8ms
image 9/9 /content/Datasets/Dataset1/test/images/CASO_87.png: 1024x800 1 ROOT, 1 TOOTH, 15.8ms
Speed: 8.3ms preprocess, 16.5ms inference, 2.6ms postprocess per image at shape (1, 3, 1024, 800)
Results saved to runs/segment/predict2
9 label

In [ ]:
from collections import defaultdict

def postprocesar_etiquetas(directorio_etiquetas = "/content/runs/segment/predict/labels"):
    """
    Objetivo:
        Permite postprocesar las etiquetas obtenidas de las predicciones realizadas para quedarse sólo con la información más relevante,
            en este caso, el contorno que tenga mayor longitud para cada clase.

    Parámetros:
        directorio_etiquetas (str): ruta de los archivos que contienen las etiquetas en formato (.txt), Por defecto adquiere el valor de
                                    "/content/runs/segment/predict/labels".

    Returns:
        resultados (dict): diccionario con sólo un conjunto de coordenadas para cada clase.
    """
    resultados = {}
    for archivo in os.listdir(directorio_etiquetas):
        if archivo.endswith('.txt'):
            ruta_archivo = os.path.join(directorio_etiquetas, archivo)
            with open(ruta_archivo, 'r') as fichero:
                lineas = fichero.readlines()
            clases = defaultdict(lambda: (None, -1))
            for linea in lineas:
                partes = linea.strip().split()
                clase = partes[0]
                coordenadas = partes[1:]
                cantidad_coordenadas = len(coordenadas) // 2
                if cantidad_coordenadas > clases[clase][1]:
                    clases[clase] = (linea.strip(), cantidad_coordenadas)
            resultados[archivo] = [coordenadas for coordenadas, cantidad in clases.values()]
    return resultados

def guardar_predicciones_postprocesadas(directorio_salida="predicciones_postprocesadas"):
    """
    Objetivo:
        Permite almacenar las predicciones postprocesadas en archivos con formato (.txt).

    Parámetros:
        directorio_salida (str): ruta para almacenar las coordenadas en formato (.txt).

    Returns:
        None: no devuelve nada.
    """
    os.makedirs(directorio_salida, exist_ok=True)
    resultados = postprocesar_etiquetas()
    for nombre_archivo, lista_coordenadas in resultados.items():
        ruta_salida = os.path.join(directorio_salida, nombre_archivo)
        with open(ruta_salida, 'w') as fichero_texto:
            for coordenadas in lista_coordenadas:
                fichero_texto.write(coordenadas + '\n')

guardar_predicciones_postprocesadas()

In [ ]:
shutil.make_archive(f'/content/predicciones_postprocesadas_final', 'zip', '/content/predicciones_postprocesadas')

'/content/predicciones_postprocesadas_final.zip'

In [ ]:
files.download(f'/content/predicciones_postprocesadas_final.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>